## Portfolio Optimization Example

Consider the following optimization problem over the portfolio $w\in \mathbb{R}^n$. The return $r$ has expected value $\alpha$ and covariance $\Sigma \in \mathbb{S}_{++}^n$. The risk aversion factor is denoted by $\gamma \geq 0$, and transaction (short-selling) cost is denoted by $\kappa_\mathrm{tc(sh)} \geq 0$:

\begin{equation}
\begin{array}{ll}
\text{maximize} \quad &\alpha^T w - \gamma w^T \Sigma w - \kappa_\mathrm{tc}^T |w-w^\mathrm{prev}| + \kappa_\mathrm{sh}^T (w)_- \\
\text{subject to} \quad &\mathbb{1}^T w = 1 \\
&\Vert w \Vert_1 \leq L
\end{array}
\end{equation}

The previous portfolio is $w^\mathrm{prev}$, $(\cdot)_-$ represents the argument's negative part, and $L \geq 1$ is the maximum value of total long positions plus the total magnitude of short positions.

With the factor model $\Sigma = F \Sigma^f F^T + D$ \[1\], we can rewrite an equivalent and [DPP-compliant](https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming) problem:

\begin{equation}
\begin{array}{ll}
\text{maximize} \quad &a^T w - \Vert \left(\Sigma^f\right)^{1/2} f \Vert_2^2 - \Vert D^{1/2} w\Vert_2^2 - k_\mathrm{tc}^T |\Delta w| + k_\mathrm{sh}^T (w)_- \\
\text{subject to} &f = F^T w \\
&\mathbb{1}^T w = 1 \\
&\Vert w \Vert_1 \leq L \\
&\Delta w = w-w^\mathrm{prev} \\
\end{array}
\end{equation}

parameterized by:

\begin{equation}
\begin{array}{ll}
a = \frac{\alpha}{\gamma} \\
F \\
\left(\Sigma^f\right)^{1/2} \\
D^{1/2} \\
k_\mathrm{tc} = \frac{\kappa_\mathrm{tc}}{\gamma} \\
k_\mathrm{sh} = \frac{\kappa_\mathrm{sh}}{\gamma} \\
w^\mathrm{prev} \\
L
\end{array}
\end{equation}

Note that we divided the objective function by the risk aversion factor $\gamma$. This way, updating the value of $\gamma$ only affects the linear part of the objective function, avoiding to compute a matrix factorization when solving the problem repeatedly.

Let's define the corresponding CVXPY problem:

In [7]:
import cvxpy as cp
import numpy as np

# define dimensions
n, m = 100, 10

# define variables
w = cp.Variable(n, name='w')
delta_w = cp.Variable(n, name='delta_w')
f = cp.Variable(m, name='f')

# define parameters
a = cp.Parameter(n, name='a')
F = cp.Parameter((n, m), name='F')
Sig_f_sqrt = cp.Parameter((m, m), name='Sig_f_sqrt')
D_sqrt = cp.Parameter((n, n), diag=True, name='D_sqrt')
k_tc = cp.Parameter(n, nonneg=True, name='k_tc')
k_sh = cp.Parameter(n, nonneg=True, name='k_sh')
w_prev = cp.Parameter(n, name='w_prev')
L = cp.Parameter(nonneg=True, name='L')

# define objective
objective = cp.Maximize(a@w
                        -cp.sum_squares(Sig_f_sqrt@f)
                        -cp.sum_squares(D_sqrt@w)
                        -k_tc@cp.abs(delta_w)
                        +k_sh@cp.minimum(0, w))

# define constraints
constraints = [f == F.T@w,
               np.ones(n)@w == 1, 
               cp.norm(w, 1) <= L, 
               delta_w == w-w_prev]

# define problem
problem = cp.Problem(objective, constraints)

Assign parameter values and solve the problem.

In [ ]:
np.random.seed(0)
gamma = 10
alpha = 1000*(-1+2*np.random.rand(n))
kappa_tc = 10*np.ones(n)
kappa_sh = 20*np.ones(n)

a.value = alpha/gamma
F.value = np.random.randn(n, m)
Sig_f_sqrt.value = np.diag(np.ones(m))
D_sqrt.value = np.diag(np.ones(n))
k_tc.value = kappa_tc/gamma
k_sh.value = kappa_sh/gamma
w_prev.value = np.zeros(n)
L.value = 1.6

val = problem.solve()

Generating C source for the problem is as easy as:

In [ ]:
import sys
sys.path.append('../')
import cvxpygen as cpg

cpg.generate_code(problem, code_dir='portfolio_code')

Now, you can use a python wrapper around the generated code as a custom CVXPY solve method:

In [ ]:
from portfolio_code.cpg_solver import cpg_solve
import numpy as np
import pickle
import time

# load the serialized problem formulation
with open('portfolio_code/problem.pickle', 'rb') as f:
    prob = pickle.load(f)

# assign parameter values
np.random.seed(0)
prob.param_dict['a'].value = alpha/gamma
prob.param_dict['F'].value = np.random.randn(n, m)
prob.param_dict['Sig_f_sqrt'].value = np.diag(np.ones(m))
prob.param_dict['D_sqrt'].value = np.diag(np.ones(n))
prob.param_dict['k_tc'].value = kappa_tc/gamma
prob.param_dict['k_sh'].value = kappa_sh/gamma
prob.param_dict['w_prev'].value = np.zeros(n)
prob.param_dict['L'].value = 1.6

# solve problem conventionally
t0 = time.time()
# CVXPY chooses eps_abs=eps_rel=1e-5, max_iter=10000, polish=True by default,
# however, we choose the OSQP default values here, as they are used for code generation as well
val = prob.solve(eps_abs=1e-3, eps_rel=1e-3, max_iter=4000, polish=False)
t1 = time.time()
print('\nPython solve time:', 1000*(t1-t0), 'ms')
print('Python objective function value:', val)

# solve problem with C code via python wrapper
prob.register_solve('CPG', cpg_solve)
t0 = time.time()
val = prob.solve(method='CPG')
t1 = time.time()
print('\nC solve time:', 1000*(t1-t0), 'ms')
print('C objective function value:', val)

### References

\[1\] Victor Ng, Robert F. Engle, Michael Rothschild,
A multi-dynamic-factor model for stock returns,
Journal of Econometrics,
Volume 52, Issues 1–2,
1992,
Pages 245-266,
ISSN 0304-4076,
https://doi.org/10.1016/0304-4076(92)90072-Y.
(https://www.sciencedirect.com/science/article/pii/030440769290072Y)